# Binary LOS Prediction

Trying two new ideas:
1. Binary LOS (over or under median)
2. Different predictor by Acuity

In [1]:
# rm(list = ls())   # wipe workspace CLEAN

library(tidymodels)
library(dplyr)
library(tidyverse)
library(xgboost)
library(yardstick)
library(vip)
library(jsonlite)
library(glmnet)
library(ranger)
library(quantreg)

── Attaching packages ───────────────────────────────────────────────────── tidymodels 1.4.1 ──

✔ broom        1.0.11     ✔ recipes      1.3.1 
✔ dials        1.4.2      ✔ rsample      1.3.1 
✔ dplyr        1.1.4      ✔ tailor       0.1.0 
✔ ggplot2      4.0.1      ✔ tidyr        1.3.1 
✔ infer        1.0.9      ✔ tune         2.0.1 
✔ modeldata    1.5.1      ✔ workflows    1.3.0 
✔ parsnip      1.4.0      ✔ workflowsets 1.1.1 
✔ purrr        1.2.0      ✔ yardstick    1.3.2 

── Conflicts ──────────────────────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()

── Attaching core tidyverse packages ─────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ readr     2.1.6     
── Conflicts ────────────────────────────────────────────────

In [2]:
training_df <- read_csv("../../../MIMIC_ED/processed/los_training_df.csv")

head(training_df)


Rows: 421233 Columns: 15
── Column specification ───────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): gender, race, disposition, icd_code, icd_title
dbl (9): stay_id, subject_id, hadm_id, ed_los_hours, acuity, medication_coun...
lgl (1): is_admitted

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


stay_id,subject_id,hadm_id,gender,race,ed_los_hours,acuity,is_admitted,disposition,medication_count,hosp_los_hours,log_hosp_los_hours,icd_code,icd_version,icd_title
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
30000012,11714491,21562392,F,WHITE,5.616667,2,TRUE,ADMITTED,14,65.00000,4.174387,5722,9,HEPATIC ENCEPHALOPATHY
30000012,11714491,21562392,F,WHITE,5.616667,2,TRUE,ADMITTED,14,65.00000,4.174387,5990,9,URIN TRACT INFECTION NOS
30000038,13821532,26255538,F,WHITE,3.300000,3,TRUE,ADMITTED,12,45.61667,3.820273,J189,10,"PNEUMONIA, UNSPECIFIED ORGANISM"
30000039,13340997,23100190,M,WHITE,8.516667,3,TRUE,ADMITTED,4,46.76667,3.845171,80709,9,FX MULT RIBS NOS-CLOSED
30000039,13340997,23100190,M,WHITE,8.516667,3,TRUE,ADMITTED,4,46.76667,3.845171,8054,9,FX LUMBAR VERTEBRA-CLOSE
30000039,13340997,23100190,M,WHITE,8.516667,3,TRUE,ADMITTED,4,46.76667,3.845171,8052,9,FX DORSAL VERTEBRA-CLOSE


## Median Hospital LOS is 56.45, we'll use this as our binary threshold

In [9]:
median(training_df$hosp_los_hours, na.rm = TRUE)

[1] 56.45

In [11]:
training_df <- training_df %>%
  mutate(
    los_binary = factor(
      hosp_los_hours > median(hosp_los_hours, na.rm = TRUE),
      levels = c(FALSE, TRUE),
      labels = c("short", "long")
    )
  )

### Train Test Split

In [14]:
set.seed(123)

data_split <- initial_split(training_df, prop = 0.8)
train <- training(data_split)
val  <- testing(data_split)


In [10]:
simplify_race <- function(r) {
  r <- toupper(as.character(r))
  
  if (grepl("WHITE", r)) {
    "WHITE"
  } else if (grepl("BLACK", r) || grepl("AFRICAN", r)) {
    "BLACK"
  } else if (grepl("HISPANIC", r) || grepl("LATINO", r)) {
    "HISPANIC"
  } else if (grepl("ASIAN", r)) {
    "ASIAN"
  } else if (grepl("NATIVE", r) || grepl("PACIFIC", r) || grepl("ALASKA", r)) {
    "NATIVE/PI"
  } else {
    "OTHER"
  }
}

# Vectorized version for use inside recipes
simplify_race_vec <- function(r) {
  sapply(r, simplify_race)
}

In [15]:
los_recipe <- recipe(los_binary ~ acuity + race + gender + ed_los_hours + medication_count, 
                     data = train) %>%
  
  # Simplify race and convert to factor
  step_mutate(
    race = factor(simplify_race_vec(race))
  ) %>%
  
  
  # Interaction features
  step_mutate(
    acuity_ed = acuity * ed_los_hours,
    meds_per_hour = medication_count / (ed_los_hours + 0.01)
  ) %>%
  
  # Dummy encode
  step_dummy(all_nominal_predictors()) %>%
  
  # Remove zero-variance predictors
  step_zv(all_predictors())


Using XGBOOST for our predictor

In [20]:
xgb_spec <- boost_tree(
  trees = 500,
  tree_depth = 6,
  learn_rate = 0.05,
  loss_reduction = 1,
  sample_size = 0.8
) %>%
  set_engine("xgboost") %>%
  set_mode("classification")


In [21]:
los_wf_bin <- workflow() %>%
  add_recipe(los_recipe) %>%
  add_model(xgb_spec)   # or log_spec


Fit the Model

In [22]:
los_fit_bin <- fit(los_wf_bin, data = train)


Predict

In [23]:
pred_class <- predict(los_fit_bin, new_data = val, type = "class")
pred_prob  <- predict(los_fit_bin, new_data = val, type = "prob")


In [24]:
preds <- val %>%
  select(los_binary) %>%
  bind_cols(pred_class, pred_prob)

head(preds)


los_binary,.pred_class,.pred_short,.pred_long
<fct>,<fct>,<dbl>,<dbl>
long,long,0.2982382,0.70176175
short,long,0.3589934,0.64100662
short,long,0.3589934,0.64100662
short,short,0.5501573,0.44984269
short,short,0.5501573,0.44984269
short,short,0.9536229,0.04637706


In [33]:
preds <- preds %>%
  mutate(
    prob_diff = abs(.pred_long - .pred_short),
    pred_conf = case_when(
      prob_diff > 0.4 ~ .pred_class,
      TRUE ~ "short"
    ) %>%
      factor(levels = c("short", "long"))
  )

In [34]:
head(preds)

los_binary,.pred_class,.pred_short,.pred_long,prob_diff,pred_conf
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>
long,long,0.2982382,0.70176175,0.4035235,long
short,long,0.3589934,0.64100662,0.2820132,short
short,long,0.3589934,0.64100662,0.2820132,short
short,short,0.5501573,0.44984269,0.1003146,short
short,short,0.5501573,0.44984269,0.1003146,short
short,short,0.9536229,0.04637706,0.9072459,short


# Evaluate

In [28]:
prop.table(table(train$los_binary))



    short      long 
0.5005371 0.4994629 

In [25]:
conf_mat(preds, truth = los_binary, estimate = .pred_class)

          Truth
Prediction short  long
     short 23154  7292
     long  18817 34984

In [26]:
metrics(preds,
        truth = los_binary,
        estimate = .pred_class) %>%
  filter(.metric %in% c("accuracy", "precision", "recall", "f_meas"))


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.6900899


In [27]:
roc_auc(preds, truth = los_binary, .pred_long)


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
roc_auc,binary,0.2438978
